In [1]:
import pandas as pd
import rdp
import numpy as np
import json

from bokeh.io import curdoc
from bokeh.plotting import ColumnDataSource, figure, output_file, show, output_notebook

from bokeh.models import HoverTool, Select, LinearColorMapper, ColorBar
from bokeh.colors import RGB

from bokeh.layouts import column, widgetbox

In [2]:
# read csv file with population data
data = pd.read_csv('./data/Export_from_Statis_BBB_with_deleted_rows.csv', dtype = str)
data.replace(to_replace = '-', value = 0, inplace=True)
data.set_index("LOR", inplace=True)

with open('./data/LOR_patches.json', 'r') as f:
    LOR = json.load(f)


LOR_areas = sorted(LOR.keys())
temp = np.interp([data.loc[k]['31.12.2017'] for k in LOR_areas],[0, 300],[255, 0])
colors = [RGB(temp[_],temp[_],temp[_]) for _ in range(len(temp))]

source = ColumnDataSource(dict(
            xs=[LOR[k]['X'] for k in LOR_areas],
            ys=[LOR[k]['Y'] for k in LOR_areas],
            name=[data.loc[k]['LOR Name'] for k in LOR_areas], 
            population=[data.loc[k]['31.12.2017'] for k in LOR_areas],
            colors=colors
                            )
                         )

# Create the figure: p
berlin_map = figure(x_axis_label='Longitude', y_axis_label='Latitude')

berlin_map.patches(xs='xs',ys='ys', fill_color='colors', source=source)

# Specify the name of the output file and show the result
output_file('berlin_map_with_bokeh_hover_no_dropdown.html')

# Create a HoverTool: hover
hover = HoverTool(tooltips=[('Area Name', '@name'), ('Population', '@population')], mode='mouse')

# Add the hover tool to the figure p
berlin_map.add_tools(hover)

show(berlin_map)